# Answers

In [108]:
import importlib
from collections import namedtuple
from box import Box
import numpy as np
from dataclasses import dataclass, field
import pandas as pd
import altair as alt
import matplotlib
import matplotlib.font_manager as font_manager
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from IPython.display import display, Markdown
from personnel_scheduling.data_structures import *
from collections import namedtuple
from pulp import *

In [17]:
def display_evaluation_results(evaluation_result):
       
    if evaluation_result.is_feasible():
        display(Markdown(f'### Feasible Solution with objective: {evaluation_result.get_total_penalty():g}'))
    else:
        display(Markdown('### Solution is infeasible. Hard rule violations:'))
        display( evaluation_result.get_df_hard_rule_violations())
    
    display(Markdown('### Grouped Penalties:'))
    display(evaluation_result.get_df_grouped_penalties())

    
def display_detailed_penalties(evaluation_result):
    display(Markdown('### Detailed Penalties:'))
    display( evaluation_result.get_df_detailed_penalties())


In [33]:
#export
def plot_activity_block(shift_index, activity_block,data):
    
    color = 'lightgrey'
    
    start_period = activity_block.get_start_period()
    number_of_periods = activity_block.get_number_of_periods()

    if activity_block.is_work():
        color = matplotlib.cm.get_cmap('tab10')(int(activity_block.activity_type))
        color = (int(color[0]*255), int(color[1]*255), int(color[2]*255)) 

    data.append({
        "shift": shift_index,
        "start": start_period+0.1,
        "end": start_period+number_of_periods-0.1,
        "id":str(color),
        "activity_type":activity_block.activity_type
    })
    
    return data


def plot_shift_schedule(shift_schedule, number_of_periods = None):
    
    if number_of_periods is None:
        number_of_periods = shift_schedule.get_number_of_periods()
        
    shift_schedule.get_number_of_periods()

    number_of_shifts =  shift_schedule.get_number_of_shifts()
    
    data = []
        
    for shift_index, shift in enumerate(shift_schedule.shifts):
        for activity_block in shift.activity_blocks:
            collection = plot_activity_block(shift_index,activity_block,data)
            
    df = pd.DataFrame(collection)

    scale = alt.Scale(domain=["(255, 127, 14)","lightgrey","(44, 160, 44)","(31, 119, 180)"],
                      range=["#ff7f0e","#d3d3d3","#2ca02c","#1f77b4"])

    bars = alt.Chart(df).mark_bar().encode(
        x='start',
        x2='end',
        y=alt.Y('shift:N'),
        color=alt.Color(
            'id:N', 
            scale=scale,
            legend=None)
    )

    text = alt.Chart(df).mark_text(
        dx=6, dy=3, color='white'
    ).encode(
        x='start',
        x2='end',
        y=alt.Y('shift:N'),
        text='activity_type:Q',
        color = alt.value('lightgrey')
    )

    chart = bars + text
    
    chart.properties(
        width=890,
        height=50
    )
    chart.display()
    
    

In [34]:
shift_schedule = ShiftSchedule.from_file("test_data/sol1.csv")
plot_shift_schedule(shift_schedule)

alt.LayerChart(...)

In [35]:
shift_schedule = ShiftSchedule.from_file("test_data/shoe_1_3_1_9_sol.csv")
plot_shift_schedule(shift_schedule)

alt.LayerChart(...)

# reporting

In [83]:
def plot_demand_line(df, ylabel, color,title, cover = None):
    
    line = alt.Chart(df).mark_line(interpolate='step-after').encode(
            x=alt.X('x',axis=alt.Axis(tickMinStep=1,title=title)),
            y=alt.X(ylabel,axis=alt.Axis(tickMinStep=1,titleFontSize=14)),
            color = color

        ).properties(
            width=800,
            height=40
        )
    
    if cover is not None:        
            
        area = alt.Chart(cover).mark_area(interpolate='step-after').encode(
            x=alt.X('x',axis=alt.Axis(tickMinStep=1,title=title)),
            y=alt.X(ylabel,axis=alt.Axis(tickMinStep=1,titleFontSize=14)),
            color = color

        ).properties(
            width=800,
            height=40
        )
        chart = line+area

    else: chart = line

    chart.display()

In [101]:
def plot_aggregate_demand(demand, cover):
    
    color = 'red'
    ylabel='aggregate demand'
    title='Aggregate Demand'
    color = alt.value(color)
    
    df = pd.DataFrame({
        'x': np.arange(len(demand)),
        'aggregate demand': demand
    })
    if cover:
        cover = pd.DataFrame({
            'x': np.arange(len(cover)),
            'aggregate demand': demand
        })
    plot_demand_line(df, ylabel, color,title, cover)
    
    
def plot_activity_demands(demands, covers=None):
    
    color_map = matplotlib.cm.get_cmap('tab10')
    
    for activity in range(len(demands)):
        
        demand = demands[activity]
        number_of_periods = len(demand)
        color = color_map(activity)
        color = (int(color[0]*255), int(color[1]*255), int(color[2]*255))
        color = alt.value('#%02x%02x%02x' % color)
        ylabel='act '+str(activity)
        title='Demand per Activity'
        
        df = pd.DataFrame({
            'x': np.arange(len(demand)),
            'act ' +str(activity): demand
        })
        
        if covers:
            cover = covers[activity]
            cover = pd.DataFrame({
                'x': np.arange(len(cover)),
                'act ' +str(activity): demand
            })
            
        else: cover = None
                
        plot_demand_line(df,ylabel, color,title, cover)


In [103]:
def plot_demands(aggregate_demand, activity_demands = None,  aggregate_cover=None, activity_covers=None):
    
    if activity_demands:
        plot_activity_demands(activity_demands, activity_covers)
        
    plot_aggregate_demand(aggregate_demand, aggregate_cover)  
    
    # if aggregate_cover is not None:
    #     plt.text(0, -5, "Note: Line corresponds to demand, colored area corresponds to cover", fontsize=16)

In [106]:
def plot_demands_for_instance(instance, shift_schedule = None):
    if shift_schedule is None:
        plot_demands(instance.get_aggregated_demand_per_period(), instance.get_activity_demands_per_period())
    else:
        plot_demands(instance.get_aggregated_demand_per_period(), instance.get_activity_demands_per_period(),  get_covered_aggregate_demand_per_period(shift_schedule, instance), get_covered_activity_demands_per_period(shift_schedule, instance))

In [105]:
def instance_report(instance):
    display(Markdown(f'## Instance: {instance.instance_name}'))
    display(Markdown(instance.get_instance_information()))
    
    display(Markdown(f'## Demand:'))
    plot_demands_for_instance(instance)

In [66]:
def instance_solution_report(instance, rule_set, shift_schedule):
    display(Markdown(f'## Instance: {instance.instance_name}'))
    display(Markdown(instance.get_instance_information()))
    display_evaluation_results(evaluate_shift_schedule(shift_schedule, rule_set))
    display(Markdown(f'## Shift Schedule:'))
    plot_shift_schedule(shift_schedule, instance.number_of_periods)
    display(Markdown(f'## Demands and Cover'))
    plot_demands_for_instance(instance, shift_schedule)

# Demmasey

## instance Class

In [68]:
class DemasseyInstance():
        
    def __init__(self, filename):
        
        self.instance_name = filename[filename.rfind('/')+1:-4]        
        
        with open(filename) as f:
            self.number_of_periods = int(f.readline())
            self.max_work_periods_per_shift = int(f.readline())
            self.number_of_days_in_horizon = int(f.readline())
            self.is_continous = int(f.readline())
            self.instance_version = int(f.readline())
            self.meaningless_parameter = int(f.readline())
            self.number_of_employees =int(f.readline()[5:])
            self.skills = ([int(n) for n in f.readline()[:-1].split()])
            self.number_of_activities = int(f.readline()[4:])
            
            line_ints = [int(n) for n in f.readline()[:-1].split()]
        
            self.min_cons_periods = line_ints[0]
            self.max_cons_periods = line_ints[1]
            self.cost_per_activity_assignment = line_ints[2]
 
            self.activity_data = []
     
            for act in range(self.number_of_activities):
                self.activity_data.append( Box() )
                self.activity_data[act].demand = ([int(n) for n in f.readline()[:-1].split()])
                self.activity_data[act].over_covering_cost = ([int(n) for n in f.readline()[:-1].split()])
                self.activity_data[act].under_covering_cost = ([int(n) for n in f.readline()[:-1].split()])
                self.activity_data[act].demand_2 = ([int(n) for n in f.readline()[:-1].split()])
                f.readline()

                
    def get_aggregated_demand_per_period(self):
        demand_per_period = []
        for p in range(self.number_of_periods):
            demand = 0
            for act in range(self.number_of_activities):
                demand = demand + self.activity_data[act].demand[p]

            demand_per_period.append(demand)

        return demand_per_period

    def get_activity_demands_per_period(self):
        demands = []

        for activity in range(self.number_of_activities):
            demands.append(self.activity_data[activity].demand)

        return demands

        
    def get_average_work_hours(self):
        return (np.sum(self.get_aggregated_demand_per_period()) / self.number_of_employees) / 4
        
    def get_instance_information(self):
        return f'Info: Activities: {self.number_of_activities}, Employees: {self.number_of_employees}, ⌀ Workhours: {self.get_average_work_hours():.02f}'   
        

In [69]:
## This is some old code, right now just kept for reference

## read instance
def read_instance(filename):

    
    with open(filename) as f:
        instance =  Box({'number_of_periods' : int(f.readline()), \
                         'max_work_periods_per_shift' : int(f.readline()), \
                         'number_of_days_in_horizon' : int(f.readline()), \
                         'is_continous' : int(f.readline()), \
                         'instance_version' : int(f.readline()), \
                         'meaningless_parameter' : int(f.readline()), \
                         'number_of_employees' : int(f.readline()[5:]), \
                         'skills' : ([int(n) for n in f.readline()[:-1].split()]), \
                         'number_of_activities' : int(f.readline()[4:])
        })

      
        line_ints = [int(n) for n in f.readline()[:-1].split()]
        
        instance.min_cons_periods = line_ints[0]
        instance.max_cons_periods = line_ints[1]
        instance.cost_per_activity_assignment = line_ints[2]
 
        instance.activity_data = []
        for act in range(instance.number_of_activities):
            instance.activity_data.append( Box())
            instance.activity_data[act].demand = ([int(n) for n in f.readline()[:-1].split()])
            instance.activity_data[act].over_covering_cost = ([int(n) for n in f.readline()[:-1].split()])
            instance.activity_data[act].under_covering_cost = ([int(n) for n in f.readline()[:-1].split()])
            instance.activity_data[act].demand_2 = ([int(n) for n in f.readline()[:-1].split()])
            f.readline()
        return instance

    def read_instance_demassey_new(filename):
        with open(filename) as f:
            instance =  Box({'number_of_periods' : 96, \
                             'max_work_periods_per_shift' : 32, \
                             'number_of_days_in_horizon' : 1, \
                             'is_continous' : 1, \
                             'instance_version' : 1, \
                             'meaningless_parameter' : 1, \
                             'number_of_employees' : int(f.readline()[5:]), \
                             'skills' : ([int(n) for n in f.readline()[:-1].split()]), \
                             'number_of_activities' : int(f.readline()[4:])
            })


            line_ints = [int(n) for n in f.readline()[:-1].split()]

            instance.min_cons_periods = line_ints[0]
            instance.max_cons_periods = line_ints[1]
            instance.cost_per_activity_assignment = line_ints[2]

            instance.activity_data = []
            for act in range(instance.number_of_activities):
                instance.activity_data.append( Box())
                instance.activity_data[act].demand = ([int(n) for n in f.readline()[:-1].split()])
                instance.activity_data[act].over_covering_cost = ([20 for n in range(96)])
                instance.activity_data[act].under_covering_cost = ([100 for n in range(96)])
                f.readline()
            return instance


In [89]:
instance = DemasseyInstance("./instances/demassey/shoe_1_3_1_9.txt")

instance_report(instance)

## Instance: shoe_1_3_1_9

Info: Activities: 3, Employees: 6, ⌀ Workhours: 6.33

## Demand:

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

## Ruleset

In [90]:
class RuleSet:
    
    def __init__(self, instance):
        self.instance = instance
    
    def check_activity_block_rules(self, activity_block):
        
        evaluation_result = RuleEvaluationResult()
        
        number_of_periods = activity_block.get_number_of_periods()
        
        

        # if it is a work block: minimum activity
        if activity_block.is_work():
            
            evaluation_result.add_penalty('ActivityCostPerPeriod', activity_block.get_number_of_work_periods() * self.instance.cost_per_activity_assignment )

            
            if number_of_periods < self.instance.min_cons_periods:
                evaluation_result.add_hard_rule_violation('MinConsPeriodsActivity')
                
            elif number_of_periods > self.instance.max_cons_periods:
                evaluation_result.add_hard_rule_violation('MaxConsPeriodsActivity')
        else:
            if number_of_periods not in [1, 4]:
                evaluation_result.add_hard_rule_violation('EitherShortOrLunchBreak')

            
        return evaluation_result
        
    def check_work_block_rules(self, work_block, shift_index = -1):
        # between two different activities we need a break
        
        evaluation_result = RuleEvaluationResult()
        
        prev_block = work_block.activity_blocks[0]
        
        for block in work_block.activity_blocks[1:]:
            if prev_block.activity_type != block.activity_type:
                return evaluation_result.add_hard_rule_violation('BreakBetweenActivitiesNeeded')
            
            prev_block = block
        
        return evaluation_result
    
    def check_shift_rules(self, shift, shift_index = -1):  
        # for this type of problem, we need to ensure that
                    
        # 1. between two different activities we need a break
        # 2. between two breaks, there needs to be work.
        
        evaluation_result = RuleEvaluationResult()
        
        short_breaks = []
        lunch_breaks = []
        
    
        hard_rule_violations = []
                
        prev_block = shift.work_and_break_blocks[0]    
        number_of_work_periods = prev_block.get_number_of_work_periods()
        
        for block in shift.work_and_break_blocks[1:]:
            if prev_block.is_work() == block.is_work():
                evaluation_result.add_hard_rule_violation('BreakAndWorkBlocksNeedToAlternate')
            
            #oberve: given the activity block checks, we know that we can only have 1 and 4 period-breaks
            if not block.is_work():
                if block.get_number_of_break_periods() == 1:
                    short_breaks.append(block)
                else:
                    lunch_breaks.append(block)
               
         
            number_of_work_periods += block.get_number_of_work_periods()
            
            prev_block = block
            
        # now check:
        if number_of_work_periods < 6*4:
            if not (len(short_breaks)==1 and len(lunch_breaks) == 0):
                evaluation_result.add_hard_rule_violation('OneShortBreakOnlyInShortShift')
        elif number_of_work_periods <= self.instance.max_work_periods_per_shift:                
            if not (len(short_breaks)==2 and len(lunch_breaks) == 1):
                    evaluation_result.add_hard_rule_violation('TwoShortBreakAndOneLunchInLongShift')
        else:
            evaluation_result.add_hard_rule_violation('MaxWorkPeriodsPerShift')
        
        return evaluation_result
    
    
    def check_demand_coverage(self, p, activity, covered_demand):
        evaluation_result = RuleEvaluationResult()
        
        difference = covered_demand - self.instance.activity_data[activity].demand[p]
        
        
        if difference > 0: ## over-covering
             evaluation_result.add_penalty('OverCovering',difference * self.instance.activity_data[activity].over_covering_cost[p])
        elif difference < 0:
            evaluation_result.add_penalty('UnderCovering',(-1)*difference * self.instance.activity_data[activity].under_covering_cost[p])
        
        return evaluation_result

In [91]:
rule_set = RuleSet(instance)

shift_schedule = ShiftSchedule.from_file("test_data/shoe_1_3_1_9_sol.csv")

evaluation_result = evaluate_shift_schedule(shift_schedule, rule_set)

display_evaluation_results(evaluation_result)

### Feasible Solution with objective: 2450

### Grouped Penalties:

rule,penalty
ActivityCostPerPeriod,2250
UnderCovering,200


In [107]:
instance_solution_report(instance, rule_set, shift_schedule)

## Instance: shoe_1_3_1_9

Info: Activities: 3, Employees: 6, ⌀ Workhours: 6.33

### Feasible Solution with objective: 2450

### Grouped Penalties:

rule,penalty
ActivityCostPerPeriod,2250
UnderCovering,200


...

## Demands and Cover

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

# Code for the Dahmen instances

> API details.


## instance Class

In [109]:
class DahmenInstance():
        
    def __init__(self, filename):
        
        self.instance_name = filename[filename.rfind('/')+1:-4]
        
        with open(filename) as f:

            self.period = int(f.readline()[7:])
            self.number_of_periods = int(f.readline()[14:])
            self.number_of_days_in_horizon = int(f.readline()[8:])
            self.number_of_employee_types = int(f.readline()[16:])
            self.flex_type_activities = int(f.readline()[6:])
            self.demand_profile_type = int(f.readline()[6:])
            self.typ_param = int(f.readline()[9:])
            self.interval_shift_start = int(f.readline()[3:])
            self.number_of_shift_types = int(f.readline()[14:])

            self.shift_type_data = []

            for shift_type in range(self.number_of_shift_types):
                line = f.readline()[:-1].split()
                shift_data = Box({'index' : int(line[1]), \
                                  'min_length_pre' : int(line[3]), \
                                  'max_length_pre' : int(line[5]), \
                                  'break_length' : int(line[7]), \
                                  'min_length_post' : int(line[9]), \
                                  'max_length_post' : int(line[11]), \
                                  'min_length' : int(line[13]), \
                                  'max_length' : int(line[15]), \
                                  'number_of_starts' : int(line[17]), \
                                  'starts' : [int(n) for n in line[19:]]})

                self.shift_type_data.append(shift_data)

            # sort of a hack: here, we assume that shift length only varies with break length
            self.break_length_to_shift_data = {}

            for shift_data in self.shift_type_data:
                self.break_length_to_shift_data[shift_data.break_length] = shift_data


            self.number_of_activities = int(f.readline()[8:])            

            min_length = self.number_of_periods

            max_length = 0

            self.activity_data = []
            for act in range(self.number_of_activities):
                line = f.readline()[:-1].split()
                activity_data = Box({'index' : int(line[1]), \
                                     'min_length' : int(line[3]), \
                                     'max_length' : int(line[5]), \
                                     'demand' : [int(n) for n in line[7:]]})

                self.activity_data.append( activity_data )

                min_length = min(min_length, activity_data.min_length)
                max_length = max (max_length, activity_data.max_length)

            self.min_length_activities = min_length
            self.max_length_activities = max_length

            self.is_cyclical = False
        
    def get_aggregated_demand_per_period(self):
        demand_per_period = []
        for p in range(self.number_of_periods):
            demand = 0
            for act in range(self.number_of_activities):
                demand = demand + self.activity_data[act].demand[p]

            demand_per_period.append(demand)

        return demand_per_period

    def get_activity_demands_per_period(self):
        demands = []

        for activity in range(self.number_of_activities):
            demands.append(self.activity_data[activity].demand)

        return demands   
    
    def get_instance_information(self):
        return f'Info: Activities: {self.number_of_activities}, Periods: {self.number_of_periods}, Shift Types: {self.number_of_shift_types}, Flex Type: {self.flex_type_activities}'  
        
def get_next_period(p,instance):
        
    if (instance.is_cyclical and p >= instance.number_of_periods-1):
        return 0
    return p+1

In [110]:
def read_instance(filename):
    with open(filename) as f:
        instance =  Box({'period' : int(f.readline()[7:]), \
                         'number_of_periods' : int(f.readline()[14:]), \
                         'number_of_days_in_horizon' : int(f.readline()[8:]), \
                         'number_of_employee_types' : int(f.readline()[16:]), \
                         'flex_type_activities' : int(f.readline()[6:]), \
                         'demand_profile_type' : int(f.readline()[6:]), \
                         'typ_param' : int(f.readline()[9:]), \
                         'interval_shift_start' : int(f.readline()[3:]),\
                         'number_of_shift_types' : int(f.readline()[14:])
        })

        instance.shift_type_data = []
        
        for shift_type in range(instance.number_of_shift_types):
            line = f.readline()[:-1].split()
            shift_data = Box({'index' : int(line[1]), \
                              'min_length_pre' : int(line[3]), \
                              'max_length_pre' : int(line[5]), \
                              'break_length' : int(line[7]), \
                              'min_length_post' : int(line[9]), \
                              'max_length_post' : int(line[11]), \
                              'min_length' : int(line[13]), \
                              'max_length' : int(line[15]), \
                              'number_of_starts' : int(line[17]), \
                              'starts' : [int(n) for n in line[19:]]})
            
            instance.shift_type_data.append(shift_data)

        # sort of a hack: here, we assume that shift length only varies with break length
        instance.break_length_to_shift_data = {}
        
        for shift_data in instance.shift_type_data:
            instance.break_length_to_shift_data[shift_data.break_length] = shift_data
            

        instance.number_of_activities = int(f.readline()[8:])            

        min_length = instance.number_of_periods

        max_length = 0
        
        instance.activity_data = []
        for act in range(instance.number_of_activities):
            line = f.readline()[:-1].split()
            activity_data = Box({'index' : int(line[1]), \
                                 'min_length' : int(line[3]), \
                                'max_length' : int(line[5]), \
                                'demand' : [int(n) for n in line[7:]]})
            
            instance.activity_data.append( activity_data )

            min_length = min(min_length, activity_data.min_length)
            max_length = max (max_length, activity_data.max_length)

        instance.min_length_activities = min_length
        instance.max_length_activities = max_length
        
        instance.is_cyclical = False
        
        return instance
    
def get_next_period(p,instance):
        
    if (instance.is_cyclical and p >= instance.number_of_periods-1):
        return 0
    return p+1

## read instance
      
    
def get_aggregated_demand_per_period(instance):
    demand_per_period = []
    for p in range(instance.number_of_periods):
        demand = 0
        for act in range(instance.number_of_activities):
            demand = demand + instance.activity_data[act].demand[p]

        demand_per_period.append(demand)

    return demand_per_period

def get_activity_demands_per_period(instance):
    demands = []
    
    for activity in range(instance.number_of_activities):
        demands.append(instance.activity_data[activity].demand)
        
    return demands

In [111]:
instance = DahmenInstance("./instances/dahmen/Implicit_Per15_Act3_prAct1_Cur2_Typ1_HD2.txt")

instance_report(instance)

## Instance: Implicit_Per15_Act3_prAct1_Cur2_Typ1_HD2

Info: Activities: 3, Periods: 96, Shift Types: 1, Flex Type: 1

## Demand:

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

## Rule sets

In [112]:
class RuleSet:
    
    def __init__(self, instance):
        self.instance = instance
     
    
    def check_activity_block_rules(self, activity_block):
        
        evaluation_result = RuleEvaluationResult()
        
        number_of_periods = activity_block.get_number_of_periods()
        
        

        # if it is a work block: minimum activity
        if activity_block.is_work():
            
            evaluation_result.add_penalty('ActivityCostPerPeriod', activity_block.get_number_of_work_periods())

            if number_of_periods < self.instance.activity_data[activity_block.activity_type].min_length:
                evaluation_result.add_hard_rule_violation('MinConsPeriodsActivity')
                
            elif number_of_periods > self.instance.activity_data[activity_block.activity_type].max_length:
                evaluation_result.add_hard_rule_violation('MaxConsPeriodsActivity')
        else:
            if number_of_periods not in self.instance.break_length_to_shift_data:
                evaluation_result.add_hard_rule_violation('WrongBreakLength')

            
        return evaluation_result
        
    def check_work_block_rules(self, work_block, shift_index = -1):
   
        evaluation_result = RuleEvaluationResult()
        
        prev_block = work_block.activity_blocks[0]
        
        if prev_block is None:
            print ("Prev Block None in shift ", shift_index)
        
        
        # no two different activities after another in one work block
        for block in work_block.activity_blocks[1:]:
            if block is None:
                print ("Block None in shift ", shift_index)
        
            if prev_block.activity_type == block.activity_type:
                evaluation_result.add_hard_rule_violation('TwoConsecutiveActivityBlocksSameActivity')
            
            prev_block = block
        
        
        ## length:
        if work_block.get_number_of_periods() < self.instance.shift_type_data[0].min_length_pre:            
            evaluation_result.add_hard_rule_violation('WorkBlockTooShort')
            
        elif work_block.get_number_of_periods() > self.instance.shift_type_data[0].max_length_pre:
            evaluation_result.add_hard_rule_violation('WorkBlockTooLong')
        
        return evaluation_result
    
    def check_shift_rules(self, shift, shift_index = -1):         
        

        
        evaluation_result = RuleEvaluationResult()
        
    
        hard_rule_violations = []
        
        # shifts are only allowed to start at certain points in time
        if shift.work_and_break_blocks[0].get_start_period() not in self.instance.shift_type_data[0].starts:
            evaluation_result.add_hard_rule_violation('OnlyStartInGivenPeriods') 

        
                
        prev_block = shift.work_and_break_blocks[0]    
        
        number_of_work_periods = prev_block.get_number_of_work_periods()
        
        number_of_break_periods = prev_block.get_number_of_break_periods()
        
        number_of_total_periods = prev_block.get_number_of_periods()
        
        for block in shift.work_and_break_blocks[1:]:
            if prev_block.is_work() == block.is_work():
                evaluation_result.add_hard_rule_violation('BreakAndWorkBlocksNeedToAlternate')
            
            #oberve: given the activity block checks, we know that we can only have 1 and 4 period-breaks
            if not block.is_work():
                number_of_break_periods += block.get_number_of_break_periods()
               
         
            number_of_work_periods += block.get_number_of_work_periods()
            number_of_total_periods += block.get_number_of_periods()
            
            prev_block = block
            
        # now check:
        if number_of_total_periods < self.instance.break_length_to_shift_data[number_of_break_periods].min_length:
            evaluation_result.add_hard_rule_violation(f'Shift {shift_index} too short: {number_of_total_periods} < {self.instance.break_length_to_shift_data[number_of_break_periods].min_length}')
            
        if number_of_total_periods > self.instance.break_length_to_shift_data[number_of_break_periods].max_length:
            evaluation_result.add_hard_rule_violation(f'Shift {shift_index } too long: {number_of_total_periods} < {self.instance.break_length_to_shift_data[number_of_break_periods].max_length}')
        
        return evaluation_result
    
    
    def check_demand_coverage(self, p, activity, covered_demand):
        evaluation_result = RuleEvaluationResult()
        
        difference = covered_demand - self.instance.activity_data[activity].demand[p]
        

        if difference < 0:
            evaluation_result.add_hard_rule_violation(f'Undercover activity: {activity} period {p} cover {covered_demand} < demand {self.instance.activity_data[activity].demand[p]}')
        
        return evaluation_result

In [113]:
rule_set = RuleSet(instance)

shift_schedule = ShiftSchedule.from_file("test_data/Implicit_Per15_Act3_prAct1_Cur2_Typ1_HD2_sol.csv")

evaluation_result = evaluate_shift_schedule(shift_schedule, rule_set)

instance_solution_report(instance, rule_set, shift_schedule)

## Instance: Implicit_Per15_Act3_prAct1_Cur2_Typ1_HD2

Info: Activities: 3, Periods: 96, Shift Types: 1, Flex Type: 1

### Feasible Solution with objective: 2600

### Grouped Penalties:

rule,penalty
ActivityCostPerPeriod,2600


...

## Demands and Cover

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)